In [17]:
import tensorflow as tf
import numpy as np
import os
import random
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [9]:
dataset_path = 'Dataset/'
augmented_dataset_path = 'Dataset_Augmented/'

In [3]:
person_folders = [f.path for f in os.scandir(dataset_path) if f.is_dir()]

In [14]:
from PIL import Image

# Example function to preprocess and save images
def preprocess_and_save_images(dataset_path, output_path, image_size=(128, 128)):
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    folders = sorted(os.listdir(dataset_path))
    
    for folder in folders:
        folder_path = os.path.join(dataset_path, folder)
        output_folder_path = os.path.join(output_path, folder)
        os.makedirs(output_folder_path, exist_ok=True)
        
        images = os.listdir(folder_path)
        
        for image_name in images:
            image_path = os.path.join(folder_path, image_name)
            output_image_path = os.path.join(output_folder_path, image_name)
            
            # Load image, resize, and save
            image = Image.open(image_path)
            image = image.resize(image_size)
            image = image.convert('RGB')  # Ensure RGB mode
            image = np.array(image) / 255.0  # Normalize pixel values
            
            # Save the preprocessed image
            Image.fromarray((image * 255).astype(np.uint8)).save(output_image_path)

# Example usage
output_path = 'Processed_Dataset/'
preprocess_and_save_images(dataset_path, output_path)
print(f'Preprocessed images saved to {output_path}.')

Preprocessed images saved to Processed_Dataset/.


In [15]:
# Function to generate pairs of images and save them to disk
def generate_and_save_image_pairs(image_data, labels, num_pairs, output_path):
    labels_list = list(set(labels))
    label_to_indices = {label: np.where(labels == label)[0] for label in labels_list}
    
    pairs_saved = 0
    for _ in range(num_pairs):
        # Select a random label for positive pair
        positive_label = random.choice(labels_list)
        positive_index = random.choice(label_to_indices[positive_label])
        positive_image = image_data[positive_index]
        
        # Select a different label for negative pair
        negative_label = random.choice(labels_list)
        while negative_label == positive_label:
            negative_label = random.choice(labels_list)
        negative_index = random.choice(label_to_indices[negative_label])
        negative_image = image_data[negative_index]
        
        # Save positive and negative image pair
        positive_image_path = os.path.join(output_path, f'positive_pair_{pairs_saved}.png')
        negative_image_path = os.path.join(output_path, f'negative_pair_{pairs_saved}.png')
        
        Image.fromarray((positive_image * 255).astype(np.uint8)).save(positive_image_path)
        Image.fromarray((negative_image * 255).astype(np.uint8)).save(negative_image_path)
        
        pairs_saved += 1
    
    print(f'Saved {pairs_saved} pairs of images to {output_path}.')

# Example usage
output_path = 'Paired_Images/'
os.makedirs(output_path, exist_ok=True)
generate_and_save_image_pairs(image_data, labels, num_pairs=1000, output_path=output_path)


Saved 1000 pairs of images to Paired_Images/.


In [18]:
# Function to generate pairs of images and save augmented pairs to disk
def augment_and_save_image_pairs(image_data, labels, num_pairs, output_path):
    labels_list = list(set(labels))
    label_to_indices = {label: np.where(labels == label)[0] for label in labels_list}
    
    # Create an ImageDataGenerator for augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2],
        fill_mode='nearest'
    )
    
    pairs_saved = 0
    for _ in range(num_pairs):
        # Select a random label for positive pair
        positive_label = random.choice(labels_list)
        positive_index = random.choice(label_to_indices[positive_label])
        positive_image = image_data[positive_index]
        
        # Select a different label for negative pair
        negative_label = random.choice(labels_list)
        while negative_label == positive_label:
            negative_label = random.choice(labels_list)
        negative_index = random.choice(label_to_indices[negative_label])
        negative_image = image_data[negative_index]
        
        # Augment positive and negative image pairs
        augmented_positive_image = datagen.random_transform(positive_image)
        augmented_negative_image = datagen.random_transform(negative_image)
        
        # Save augmented positive and negative image pair
        positive_image_path = os.path.join(output_path, f'augmented_positive_pair_{pairs_saved}.png')
        negative_image_path = os.path.join(output_path, f'augmented_negative_pair_{pairs_saved}.png')
        
        Image.fromarray((augmented_positive_image * 255).astype(np.uint8)).save(positive_image_path)
        Image.fromarray((augmented_negative_image * 255).astype(np.uint8)).save(negative_image_path)
        
        pairs_saved += 1
    
    print(f'Augmented and saved {pairs_saved} pairs of images to {output_path}.')

# Example usage
output_path = 'Augmented_Paired_Images/'
os.makedirs(output_path, exist_ok=True)
augment_and_save_image_pairs(image_data, labels, num_pairs=1000, output_path=output_path)


Augmented and saved 1000 pairs of images to Augmented_Paired_Images/.
